In [73]:
import json
import requests
from src import utils
import time
import os

In [74]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [75]:
api = utils.read_json_file(api_path)

In [76]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [77]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [78]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'one-shot-cot-result-Ko-gsm8k-test.jsonl'
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-result-Ko-gsm8k-test.jsonl


In [79]:
# file_one-shot-cot-result-Ko-gsm8k-test.jsonl

In [80]:
jsonl_writer = utils.JSONLWriter(output_file)

In [81]:
data = utils.read_jsonlines(input_file)

In [82]:
len(data)

1319

In [89]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [90]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [91]:
def get_gsm8k_one_shot_cot(pre_prompt, que):
    suf_prompt = f'질문: {que} 풀이 : '
    prompt = pre_prompt + suf_prompt
    return prompt

In [95]:
PROMPT1 = "질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? \
풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. \
정답: 72 \\n"


In [94]:
print(prompt1)

질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n


In [93]:
cnt = 0
split_cnt = 0
for ins in data:
#     print(ins.keys())
#     break
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    prompt = get_gsm8k_one_shot_cot(PROMPT1, kor_que)
    

    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    ins['kor_infer'] = res_context
    
    jsonl_writer.write_json_line(ins)
    
#     print(prompt)
#     if cnt == 5:
#         break
#     else:
#         cnt += 1

/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 풀이 : 
res_context : 
한 벌의 로브를 만드는 데 필요한 섬유의 양은 로브의 길이와 원하는 텍스처에 따라 달라집니다. 로브의 길이가 2덩이이고 원하는 텍스처의 밀도가 50%라고 가정하면 로브의 총 섬유 양은 다음과 같습니다: 로브의 길이 = 2 * 50% = 8 로브의 원하는 텍스처의 밀도 = 50% * 100% = 200 섬유가 필요합니다. 로브의 총 텍스처의 양은 200 * 100% = 2400 섬유입니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 풀이 : 
res_context : 
제공된 정보만으로는 조쉬가 얼마나 많은 이익을 얻었는지 확인할 수 없습니다. 이익의 정확한 값을 계산하려면 집의 가치가 높아져야 합니다. 집의 가치는 $80,000에서 $50,000 = $150,000입니다. 조쉬는 집을 판매함으로써 150%의 이익을 얻었지만, 이는 $150,000 - $80,000 = $80,000입니다. 따라서 조쉬는 약 $80,000 - $50,000 = $80,000의 이익을 얻었습니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 매일, 웬디는 각 닭에게 씨앗, 멸치, 채소가 섞인 닭 사료 3컵씩을 주어 건강하게 유지합니다. 그녀는 닭에게 3번의 식사로 사료를 제공합니다. 아침에는 15컵의 사료를 주고, 오후에는 25컵의 사료를 또 다시 주며, 만약 웬디의 닭 떼가 20마리라면, 하루의 마지막 식사에서 닭에게 몇 컵의 사료를 주어야 할까요? 풀이 : 
res_context : 
웬디는 닭에게 3번의 식사로 사료를 제공합니다. 하루에 3번의 식사가 있으므로 웬디의 닭 떼 수가 20마리이고 닭이 3마리이면 웬디는 하루에 3 \* (15+25) = 180 마리의 닭을 닭 사료로 제공합니다. 닭이 20마리이고 닭 떼 수가 4마리이면 웬디는 하루에 4 \* (20-20) = 160 마리의 닭을 닭 사료로 제공합니다. 따라서 웬디는 닭에게 180 \* 4 = 2400 마리의 닭을 사료로 제공해야 합니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 카일러는 새 아파트를 위해 유리잔을 사러 가게에 갔습니다. 하나의 유리잔 가격은 $5이지만, 두 번째 유리잔부터는 가격의 60%만 지불하면 됩니다. 카일러는 16개의 유리잔을 사고 싶습니다. 그들을 위해 얼마를 지불해야 할까요? 풀이 : 
res_context : 
정답은 $16입니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 툴루즈는 찰스턴의 양의 두 배를 가지고 있습니다. 찰스턴은 시애틀의 양의 4 배를 가지고 있습니다. 시애틀은 양이 20마리 있으므로, 툴루즈, 찰스턴 및 시애틀이 함께 가지고 있는 양은 얼마입니까? 풀이 : 
res_context : 
정답은 24입니다.
----


In [13]:
# api

In [14]:
# url = api['url']

In [15]:
# headers = api['headers']

In [16]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [17]:
# headers

In [18]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [19]:
model = model_api(api['url'], api['headers'])

In [21]:
# body = json.dumps(body_temp)

In [22]:
# print(body)

In [23]:
# response = api.generate(body)

In [24]:
# context = response.json()['data'][0]['result'][0]

In [25]:
# print(context)

In [ ]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [26]:
def get_gsm8k_one_shot_cot(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [ ]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [28]:
cnt = 0
split_cnt = 0
for ins in data:
    print(ins.keys())
    break
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs(kor_que)
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 5:
        break
    else:
        cnt += 1

dict_keys(['question', 'answer', 'kor_question', 'kor_answer', 'answer_s'])
